# Library

In [1]:
import glob
import joblib

import warnings
warnings.filterwarnings('ignore')

import os
from os.path import join
import sys
path = os.path.join(os.pardir, os.pardir)
sys.path.append(path)

import numpy as np
import pandas as pd
import xarray as xr

from scipy.signal import savgol_filter
from src.data.process_data import merge_satellite, add_observation, add_weather, statedev_fill, features_modification, compute_vi

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from src.data.preprocessing import Smoother

# from datascaler import DatasetScaler

from src.constants import M_COLUMNS, S_COLUMNS, G_COLUMNS, TARGET, FOLDER, TARGET_TEST


ROOT_DIR = os.path.join(os.pardir, os.pardir)
test = False
folder = 'augment_100_5'

In [2]:
file_name = "train.nc"
if test:
    file_name = "test.nc"


# Load Satellite Dataset
if folder == "augment_100_5":
    xds = merge_satellite(file_name)
else:
    path_sat = join(ROOT_DIR, "data", "external", "satellite", folder, file_name)
    xds = xr.open_dataset(path_sat, engine="scipy")
    
xds

<xarray.Dataset>
Dimensions:    (ts_obs: 557, ts_aug: 100, state_dev: 24)
Coordinates:
  * ts_obs     (ts_obs) int32 0 1 2 3 4 5 6 7 ... 550 551 552 553 554 555 556
  * ts_aug     (ts_aug) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
  * state_dev  (state_dev) int32 23 22 21 20 19 18 17 16 15 ... 7 6 5 4 3 2 1 0
    time       (ts_obs, state_dev, ts_aug) object '2022-07-14' ... '2021-12-06'
Data variables:
    red        (ts_obs, ts_aug, state_dev) float32 nan nan nan ... nan 1.21e+03
    green      (ts_obs, ts_aug, state_dev) float32 nan nan nan ... nan 1.008e+03
    blue       (ts_obs, ts_aug, state_dev) float32 nan nan nan ... nan 676.3
    rededge1   (ts_obs, ts_aug, state_dev) float32 nan nan nan ... nan 1.512e+03
    rededge2   (ts_obs, ts_aug, state_dev) float32 nan nan nan ... nan 1.474e+03
    rededge3   (ts_obs, ts_aug, state_dev) float32 nan nan nan ... nan 1.566e+03
    nir        (ts_obs, ts_aug, state_dev) float32 nan nan nan ... nan 1.478e+03
    swir       (ts_obs, ts_aug, state_dev) float32 nan nan nan ... nan 606.1

In [3]:

def categorical_encoding(xds: xr.Dataset) -> xr.Dataset:
    # Encode Rice Crop Intensity feature D = 2 and T = 3
    xds["Rice Crop Intensity(D=Double, T=Triple)"] = (
        xds["Rice Crop Intensity(D=Double, T=Triple)"]
        .str.replace("D", "2")
        .str.replace("T", "3")
        .astype(np.int8)
    )
    return xds

file_name = "train_enriched.csv"
if test:
    file_name = "test_enriched.csv"

path = join(ROOT_DIR, "data", "interim", file_name)
# Read csv EY data
df = pd.read_csv(path, index_col=0)
# Set index name as ts_obs for linked both Dataset
df.index.name = "ts_obs"
# # Convert pandas.DataFrame into xarray.Dataset and merge on ts_obs

xds = xr.merge([xds, df.to_xarray()], compat='override')
# # Encode categoricals data
xds = categorical_encoding(xds)

In [4]:
xds = add_weather(xds)
xds['time'].isnull().any()

<xarray.DataArray 'time' ()>
array(False)

In [5]:
# Compute vegetable indices
xds = compute_vi(xds)
xds['time'].isnull().any()

<xarray.DataArray 'time' ()>
array(False)

In [6]:
xds = statedev_fill(xds)
xds['time'].isnull().any()

<xarray.DataArray 'time' ()>
array(False)

In [7]:
xds = Smoother(mode='savgol').transform(xds)
xds['time'].isnull().any()

<xarray.DataArray 'time' ()>
array(False)

In [8]:
xds['time'].isnull().any()

<xarray.DataArray 'time' ()>
array(False)

In [9]:
xds = features_modification(xds, test)
xds

<xarray.DataArray 'time' ()>
array(False)


<xarray.Dataset>
Dimensions:                                  (ts_obs: 557, ts_aug: 100,
                                              state_dev: 24, datetime: 275,
                                              name: 3)
Coordinates:
  * ts_obs                                   (ts_obs) int32 0 1 2 ... 555 556
  * ts_aug                                   (ts_aug) int64 0 1 2 3 ... 97 98 99
  * state_dev                                (state_dev) int32 23 22 21 ... 1 0
  * datetime                                 (datetime) datetime64[ns] 2021-1...
  * name                                     (name) object 'Chau_Phu' ... 'Th...
Data variables: (12/31)
    ndvi                                     (ts_obs, ts_aug, state_dev) float32 ...
    savi                                     (ts_obs, ts_aug, state_dev) float32 ...
    evi                                      (ts_obs, ts_aug, state_dev) float32 ...
    rep                                      (ts_obs, ts_aug, state_dev) float32 ...
    osavi                                    (ts_obs, ts_aug, state_dev) float32 ...
    rdvi                                     (ts_obs, ts_aug, state_dev) float32 ...
    ...                                       ...
    uvindex                                  (datetime, name) int64 4 5 ... 2 3
    moonphase                                (datetime, name) float64 0.11 .....
    solarexposure                            (datetime, name) int64 42091 ......
    time                                     (ts_obs, state_dev, ts_aug) datetime64[ns] ...
    District                                 (ts_obs) object 'Chau_Phu' ... '...
    Rice Yield (kg/ha)                       (ts_obs) float64 5.5e+03 ... 7.2...